In [1]:
!pip install transformers

     |████████████████████████████████| 1.9MB 7.4MB/s 
     |████████████████████████████████| 3.2MB 36.6MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=449a7f28068949100ad7e05f0dffe474cf5fb382e1fd7b3dd0373bd15837f883
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!pip install datasets

     |████████████████████████████████| 184kB 8.0MB/s 
     |████████████████████████████████| 112kB 26.1MB/s 
     |████████████████████████████████| 245kB 21.9MB/s 


In [3]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [4]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric

### 데이터 가져오기

In [5]:
squad_dataset = load_dataset('amazon_reviews_multi')
train = squad_dataset['train']
test = squad_dataset['test']

print(train.shape)
print(test.shape)

No config specified, defaulting to: amazon_reviews_multi/all_languages


Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/f3357bd271e187385a38574fe31b8fb10055303f67fa9fce55e84d08c4870efd. Subsequent calls will reuse this data.
(1200000, 8)
(30000, 8)


data collected btw 2015. 11. 01 ~ 2019. 11. 01

In [6]:
train[0]

{'language': 'de',
 'product_category': 'sports',
 'product_id': 'product_de_0865382',
 'review_body': 'Armband ist leider nach 1 Jahr kaputt gegangen',
 'review_id': 'de_0203609',
 'review_title': 'Leider nach 1 Jahr kaputt',
 'reviewer_id': 'reviewer_de_0267719',
 'stars': 1}

In [7]:
train = pd.DataFrame.from_dict(train)
test = pd.DataFrame.from_dict(test)
train.head()

,language,product_category,product_id,review_body,review_id,review_title,reviewer_id,stars
0,de,sports,product_de_0865382,Armband ist leider nach 1 Jahr kaputt gegangen,de_0203609,Leider nach 1 Jahr kaputt,reviewer_de_0267719,1
1,de,home_improvement,product_de_0678997,In der Lieferung war nur Ein Akku!,de_0559494,EINS statt ZWEI Akkus!!!,reviewer_de_0783625,1
2,de,drugstore,product_de_0372235,"Ein Stern, weil gar keine geht nicht. Es hande...",de_0238777,Achtung Abzocke,reviewer_de_0911426,1
3,de,drugstore,product_de_0719501,"Dachte, das wären einfach etwas festere Binden...",de_0477884,Zu viel des Guten,reviewer_de_0836478,1
4,de,toy,product_de_0022613,Meine Kinder haben kaum damit gespielt und nac...,de_0270868,Qualität sehr schlecht,reviewer_de_0736276,1


In [ ]:
train.language.unique()

array(['de', 'en', 'es', 'fr', 'ja', 'zh'], dtype=object)

In [8]:
train = train[train.language == 'en']
test = test[test.language == 'en']

print(train.shape)
print(test.shape)

(200000, 8)
(5000, 8)


In [9]:
train.head()

,language,product_category,product_id,review_body,review_id,review_title,reviewer_id,stars
200000,en,furniture,product_en_0740675,Arrived broken. Manufacturer defect. Two of th...,en_0964290,I'll spend twice the amount of time boxing up ...,reviewer_en_0342986,1
200001,en,home_improvement,product_en_0440378,the cabinet dot were all detached from backing...,en_0690095,Not use able,reviewer_en_0133349,1
200002,en,home,product_en_0399702,I received my first order of this product and ...,en_0311558,The product is junk.,reviewer_en_0152034,1
200003,en,wireless,product_en_0444063,This product is a piece of shit. Do not buy. D...,en_0044972,Fucking waste of money,reviewer_en_0656967,1
200004,en,pc,product_en_0139353,went through 3 in one day doesn't fit correct ...,en_0784379,bubble,reviewer_en_0757638,1


In [10]:
train.stars.value_counts()

5    40000
4    40000
3    40000
2    40000
1    40000
Name: stars, dtype: int64

In [ ]:
test.stars.value_counts()

3    1000
2    1000
5    1000
1    1000
4    1000
Name: stars, dtype: int64

In [11]:
train = train.sample(n=70000, random_state=1234)
train.shape

(70000, 8)

In [ ]:
train.stars.value_counts()

1    14111
4    14060
5    13990
2    13969
3    13870
Name: stars, dtype: int64

In [12]:
test.index = range(len(test))
train.index = range(len(train))

train.head()

,language,product_category,product_id,review_body,review_id,review_title,reviewer_id,stars
0,en,beauty,product_en_0888943,The comb makes it easy to apply in stripes. Th...,en_0092109,Nice,reviewer_en_0920882,4
1,en,furniture,product_en_0160639,"Not necessarily a product issue, but the box w...",en_0345397,No instructions included,reviewer_en_0740279,1
2,en,home_improvement,product_en_0335455,I used this light to replace an old fluorescen...,en_0837577,What more could you ask for?,reviewer_en_0466530,5
3,en,drugstore,product_en_0247695,Smart and sleek. The only thing that puzzles m...,en_0332740,Smart and sleek,reviewer_en_0092514,4
4,en,other,product_en_0586170,Packege arrived wet and broken,en_0996773,One Star,reviewer_en_0364924,1


In [13]:
test.head()

,language,product_category,product_id,review_body,review_id,review_title,reviewer_id,stars
0,en,apparel,product_en_0902516,"These are AWFUL. They are see through, the fab...",en_0199937,Don’t waste your time!,reviewer_en_0097389,1
1,en,other,product_en_0348072,I bought 4 and NONE of them worked. Yes I used...,en_0863335,One Star,reviewer_en_0601537,1
2,en,other,product_en_0356154,On first use it didn't heat up and now it does...,en_0565010,Totally useless,reviewer_en_0970351,1
3,en,jewelry,product_en_0583322,You want an HONEST answer? I just returned fro...,en_0963290,Gold filled earrings,reviewer_en_0216125,1
4,en,industrial_supplies,product_en_0487636,The glue works fine but the container is impos...,en_0238156,Poor container,reviewer_en_0514203,1


### train data 전처리

In [ ]:
train.head()

,language,product_category,product_id,review_body,review_id,review_title,reviewer_id,stars
0,en,beauty,product_en_0888943,The comb makes it easy to apply in stripes. Th...,en_0092109,Nice,reviewer_en_0920882,4
1,en,furniture,product_en_0160639,"Not necessarily a product issue, but the box w...",en_0345397,No instructions included,reviewer_en_0740279,1
2,en,home_improvement,product_en_0335455,I used this light to replace an old fluorescen...,en_0837577,What more could you ask for?,reviewer_en_0466530,5
3,en,drugstore,product_en_0247695,Smart and sleek. The only thing that puzzles m...,en_0332740,Smart and sleek,reviewer_en_0092514,4
4,en,other,product_en_0586170,Packege arrived wet and broken,en_0996773,One Star,reviewer_en_0364924,1


In [16]:
train['stars'] = list(map(lambda x : x - 1, train['stars']))
test['stars'] = list(map(lambda x : x - 1, test['stars']))

train.head()

,language,product_category,product_id,review_body,review_id,review_title,reviewer_id,stars
0,en,beauty,product_en_0888943,The comb makes it easy to apply in stripes. Th...,en_0092109,Nice,reviewer_en_0920882,3
1,en,furniture,product_en_0160639,"Not necessarily a product issue, but the box w...",en_0345397,No instructions included,reviewer_en_0740279,0
2,en,home_improvement,product_en_0335455,I used this light to replace an old fluorescen...,en_0837577,What more could you ask for?,reviewer_en_0466530,4
3,en,drugstore,product_en_0247695,Smart and sleek. The only thing that puzzles m...,en_0332740,Smart and sleek,reviewer_en_0092514,3
4,en,other,product_en_0586170,Packege arrived wet and broken,en_0996773,One Star,reviewer_en_0364924,0


In [ ]:
train['review_body'][0]

'The comb makes it easy to apply in stripes. The chaulk works as well as other we have tried.'

In [17]:
# 리뷰 문장 추출

sentences = train['review_body']
sentences[0]

'The comb makes it easy to apply in stripes. The chaulk works as well as other we have tried.'

In [18]:
# BERT 입력 형식에 맞게 변환

sentences = ['[CLS] ' + str(sentence) +' [SEP]' for sentence in sentences]
sentences[:10]

['[CLS] The comb makes it easy to apply in stripes. The chaulk works as well as other we have tried. [SEP]',
 '[CLS] Not necessarily a product issue, but the box was half ripped open when we recieved it, and the instructions were missing. We are not sure if any parts are missing, because of course, there is also no parts list. Going to try and find instructions somewhere [SEP]',
 "[CLS] I used this light to replace an old fluorescent security light that I had over my driveway. It was very easy to install and produces a very bright white light with a wide radius, but not bright enough to disturb the neighbors. Very sturdy construction. The waterproof gasket can be tricky to keep in during installation but if you use a little grease or glue to tack it in place, no problem! You won't be sorry! [SEP]",
 '[CLS] Smart and sleek. The only thing that puzzles me is that it gives a slightly different weigh every time u step on it. Not sure why but otherwise it serves the purpose well. [SEP]',
 '

In [19]:
labels = train['stars'].values
labels

array([3, 0, 4, ..., 4, 1, 3])

In [20]:
# Tokenizing

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print(sentences[0])
print(tokenized_texts[0])


[CLS] The comb makes it easy to apply in stripes. The chaulk works as well as other we have tried. [SEP]
['[CLS]', 'the', 'com', '##b', 'makes', 'it', 'easy', 'to', 'apply', 'in', 'strip', '##es', '.', 'the', 'cha', '##ul', '##k', 'works', 'as', 'well', 'as', 'other', 'we', 'have', 'tried', '.', '[SEP]']


In [21]:
MAX_LEN = 128 # 입력 토큰의 최대 seq 길이
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts] # 토큰을 숫자 인덱스로
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post') # max_len에 맞게 문장 자르고 모자란 부분 패딩 0으로 채움

input_ids[0]

array([  101, 10105, 10212, 10457, 20562, 10271, 44346, 10114, 48515,
       10106, 44445, 10171,   119, 10105, 18939, 10604, 10174, 14009,
       10146, 11206, 10146, 10684, 11951, 10529, 27156,   119,   102,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [22]:
attention_masks = [] # attention_mask 초기화

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [23]:
# train_test_split

train_inputs, val_inputs, train_lab, val_lab = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)

train_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_lab = torch.tensor(train_lab)
train_masks = torch.tensor(train_masks)
val_inputs = torch.tensor(val_inputs)
val_lab = torch.tensor(val_lab)
val_masks = torch.tensor(val_masks)				

print(train_inputs[0])
print(train_lab[0])
print(train_masks[0])
print(val_inputs[0])
print(val_lab[0])
print(val_masks[0])

tensor([   101,  82365,  11387, 101300,  10106,  17283,    119,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [24]:
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_lab)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks,val_lab)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

### test dat 전처리

In [ ]:
test.head()

,language,product_category,product_id,review_body,review_id,review_title,reviewer_id,stars
0,en,apparel,product_en_0902516,"These are AWFUL. They are see through, the fab...",en_0199937,Don’t waste your time!,reviewer_en_0097389,0
1,en,other,product_en_0348072,I bought 4 and NONE of them worked. Yes I used...,en_0863335,One Star,reviewer_en_0601537,0
2,en,other,product_en_0356154,On first use it didn't heat up and now it does...,en_0565010,Totally useless,reviewer_en_0970351,0
3,en,jewelry,product_en_0583322,You want an HONEST answer? I just returned fro...,en_0963290,Gold filled earrings,reviewer_en_0216125,0
4,en,industrial_supplies,product_en_0487636,The glue works fine but the container is impos...,en_0238156,Poor container,reviewer_en_0514203,0


In [25]:
sentences = test['review_body']

sentences = ['[CLS] ' + str(sent) + ' [SEP]' for sent in sentences]

labels = test['stars'].values

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

MAX_LEN = 128 
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,  13252,  10301,    138,  83695, 100673,    119,  11696,  10301,
         12888,  11222,    117,  10105,  12211,  95989,  84992,  11850,  21783,
         63143,  11143,    117,  10111,  10689,  21635,  11850,  12694,    100,
           187,  58799,    119, 101590,  10165,  11989,  12172,  57047,  10114,
         10347,  46267,  10112,  15102,    117,  10473,    146,  11639,  97309,
         39542,  15127,  15079,  13664,  10142,  11762,    119,    146,  10894,
         10115,    100,    188,  13246,  73639,  10112,  11345,  12373,  10105,
         21905,  10124,  10380,  23247,    119,    102,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [26]:
batch_size = 32

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

### 모델 생성

In [27]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [28]:
# 디바이스 설정

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [29]:
# BERT 모델 생성

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [30]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps=1e-8)

epochs = 2

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

### 모델 학습

In [31]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [32]:
# 랜덤시드 고정
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

model.zero_grad()

for epoch_i in range(0, epochs):

    print('')
    print('====== Epoch {:} / {:} ======'.format(epoch_i +1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0: # 경과표시
            elapsed = format_time(time.time() - t0)
            print(' Batch {:>5,} of {:>5,}.  Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        
        batch = tuple(t.to(device) for t in batch) # 배치를 GPU로
        b_input_ids, b_input_mask, b_labels = batch # 배치에서 데이터 추출

        # forward 수행
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask = b_input_mask,
                        labels = b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 그래디언트 클리핑
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print('')
    print(' Average training loss: {0:.2f}'.format(avg_train_loss))
    print(' Training epoch took: {:}'.format(format_time(time.time() - t0)))

    print('')
    print('Running Validation...')

    t0 = time.time()
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in val_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    print(' Accuracy: {0:.2f}'.format(eval_accuracy/nb_eval_steps))
    print(' Validation took: {:}'.format(format_time(time.time() -t0)))

print('')
print('Training Complete!')


====== Epoch 1 / 2 ======
Training...
 Batch   500 of 1,969.  Elapsed: 0:03:18.
 Batch 1,000 of 1,969.  Elapsed: 0:06:35.
 Batch 1,500 of 1,969.  Elapsed: 0:09:53.

 Average training loss: 1.10
 Training epoch took: 0:12:58

Running Validation...
 Accuracy: 0.56
 Validation took: 0:00:27

====== Epoch 2 / 2 ======
Training...
 Batch   500 of 1,969.  Elapsed: 0:03:18.
 Batch 1,000 of 1,969.  Elapsed: 0:06:35.
 Batch 1,500 of 1,969.  Elapsed: 0:09:53.

 Average training loss: 0.93
 Training epoch took: 0:12:58

Running Validation...
 Accuracy: 0.57
 Validation took: 0:00:27

Training Complete!


### 테스트셋 확인

In [33]:
t0 = time.time()

model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for step, batch in enumerate(test_dataloader):

    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    157.    Elapsed: 0:00:12.

Accuracy: 0.56
Test took: 0:00:19


### 새로운 문장 테스트

In [35]:
# 입력 데이터 변환

def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [36]:
# 문장 테스트

def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

아마존 리뷰

In [38]:
logits = test_sentences(["I bought it to read comics and it's perfect for that. Multitasking with Chrome (1-3 tabs), comixology, hoopla, gmail, and kindle unlimited is smooth. It can't play HD video. Not a problem for me but if you plan to stream you'll be disappointed."])

# 202003
# 실제로 3

print(logits)
print(np.argmax(logits))

[[-2.6845248  -0.7142221   1.0594974   2.090311    0.57804006]]
3


In [39]:
logits = test_sentences(["Maybe you just get what you pay for. This tablet is SOOOO painfully slow it's just not even practical to use it at all. Right out of the box, even tapping an icon several seconds to even acknowledge the touch! WOW. HORRID. Buy something else."])

# 202006
# 실제로 0

print(logits)
print(np.argmax(logits))

[[ 3.4428568   1.3893623  -0.19231015 -2.1154518  -2.5575473 ]]
0


In [40]:
logits = test_sentences(["Just got it today, so far I’m highly impressed by the screen quality. It’s a little heavier than I expected but the build quality is great. Don’t expect to get a great camera.Overall I definitely recommend it"])

# 202004
# 실제로 4

print(logits)
print(np.argmax(logits))

[[-3.393276   -2.624411   -0.45528054  2.9194224   3.4010327 ]]
4


월마트 리뷰

In [41]:
logits = test_sentences(["This game is awsome I saw it in stores for $49 I put In my PS5 and it worked and downloaded it thanks so much"])

# 실제로 4

print(logits)
print(np.argmax(logits))

[[-1.0467199 -1.0655137 -0.4496968  1.1735873  1.7048708]]
4


In [42]:
logits = test_sentences(["If your want to get this game plz get it on next gen or pc, on the ps4 theres constant glitches and areas take very long to load. Even pausing the game you have to wait for the pause screen to load, and anything fast past that's isn't sitting and talking to another character the frame rate drops, making it very difficult to fight or just play the game."])

# 실제로 0

print(logits)
print(np.argmax(logits))

[[ 2.1892717   2.4908571   0.72052234 -1.99862    -3.6210847 ]]
1


In [43]:
logits = test_sentences(["Case was broken. Game wouldn't play without an update. Once i got it updated, the game was still a glitchy mess. I wouldn't buy it for the PS4"])

# 실제로 1

print(logits)
print(np.argmax(logits))

[[ 2.866775   2.1264791  0.4451562 -2.092443  -3.4894063]]
0


In [44]:
logits = test_sentences(["I give cyborpunk 3stars even after the update. It's a fun game to play, great story, and definitely not suitable for children. The only issues with this game, is they rushed the mechanics on this game. Very glitchy, and has multiple holes in the game. Overall I give the game 3stars, once all the fixes are made, and updated this game would earn more stars."])

# 실제로 2

print(logits)
print(np.argmax(logits))

[[-2.473995   -0.47060424  1.3981096   2.3322906  -1.0795727 ]]
3


In [45]:
logits = test_sentences(["Very cool game has some issues but playable and patches will fix bugs you all know how gaming industry works release n repair w patches lol. Great gaming experience tho n adults only...kids. lol"])

# 실제로 3

print(logits)
print(np.argmax(logits))

[[-2.4465759  -1.5538871  -0.22631714  1.9741671   2.1106997 ]]
4


In [46]:
logits = test_sentences(["This game is awsome I saw it in stores for $49 I put In my PS5 and it worked and downloaded it thanks so much"])

# 실제로 4

print(logits)
print(np.argmax(logits))

[[-1.0467199 -1.0655137 -0.4496968  1.1735873  1.7048708]]
4


In [47]:
logits = test_sentences(["Game is laughable at how its poorly optimize it is, even on the ps5 the game crashes and no way to progress"])

# 실제로 0

print(logits)
print(np.argmax(logits))

[[ 2.3381815   2.376131    0.60586745 -2.006638   -3.4352758 ]]
1
